# Querying OpenAlex Database with author ids

In [1]:
import requests, json
import csv
import ast

In [4]:
#List of names to test
test_name_list = [
    "william pao",
    "frederick suchy",
    "malcolm cox",
    "nancy cooke",
    "allan sniderman",
    #"vincent dennis",
    #"alessandra pernis",
    #"john minna",
    #"dennis bier",
    #"roger pomerantz"
] #names from asci/aap data

In [2]:
#this cell is an example of how we are calling OpenAlex data
#-----------------TESTING--------------

author_id = "A2242856272"
# specify endpoint
endpoint = 'works'

# build the 'filter' parameter
page = 'page={}'

# put the URL together
filtered_works_url = f'https://api.openalex.org/{endpoint}?filter=author.id:{author_id}&{page}'
print(f'complete URL with filters:\n{filtered_works_url}')

complete URL with filters:
https://api.openalex.org/works?filter=author.id:A2242856272&page={}


In [3]:
#general formula to call data
#endpoint,filterInfo gives acces to table in database, look at documentation and example below
base_url = 'https://api.openalex.org/'
def set_query_call_name(endpoint, filterInfo, name):
    fullquery = base_url+endpoint+'?'+filterInfo+name
    response = requests.get(fullquery)
    visualize_data = response.json()
    return visualize_data
    #print(json.dumps(visualize_data, indent=2)

In [7]:
#finds all work ids by author ids 
def work_id(givenAuthorID):
        page = 'page={}'
        filtered_works_url = f'https://api.openalex.org/works?filter=author.id:{givenAuthorID}&{page}'
        page = 1
        has_more_pages = True
        all_worksID = set()

        # loop through pages
        while has_more_pages:

            # set page value and request page from OpenAlex
            url = filtered_works_url.format(page)
            page_with_results = requests.get(url).json()

            # loop through partial list of results
            results = page_with_results['results']
            for i,work in enumerate(results):
                openalex_id = work['id'].replace("https://openalex.org/", "")
                all_worksID.add(openalex_id)
            # next page
            page += 1

            # end loop when either there are no more results on the requested page 
            # or the next request would exceed 15 results
            per_page = page_with_results['meta']['per_page']
            has_more_pages = len(results) == per_page
        return (all_worksID)
#list_output = work_id('A5044648110')
#print(f'{len(list_output)}')

In [8]:
#gets each publications by its workid
def findWork(workId):
    fullquery = base_url+'works/'+workId
    response = requests.get(fullquery)
    visualize_data = response.json()
    visualize_data.pop("abstract_inverted_index")
    visualize_data.pop("related_works")
    visualize_data.pop("ngrams_url")
    #clean the unicode
    #visualize_data[""]
    return visualize_data
#findWork('W2044177650')
#dont need abstract_inverted_index,related_works data

In [11]:
#opens a csv with name and associated ids, then puts it into a dictionary
#ie. {"alan d'andrea": ['A5045114091', 'A5044648110'], 'james fagin': ['A5080716045', 'A5026026919', 'A5079286906']}
names = {}

with open('1995_2010.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  
    for row in reader:
        name = row[0]
        openIds_str = row[1]  
        openIds_list = ast.literal_eval(openIds_str)
        names[name] = openIds_list

print(names)


SyntaxError: EOL while scanning string literal (<unknown>, line 1)

In [19]:
#combined work_id(authorID) and findWork(workID) to find publications by author ids saved in a separte csv file

#testing = {"alan d'andrea": ['A5045114091', 'A5044648110'], 'james fagin': ['A5080716045', 'A5026026919', 'A5079286906']}

personDict = {}
count = 0
error_authors = []

for name, openIds in names.items():
    count += 1
    print(f'{count}. Working to find works by: {name}')
    tempList = []
    for authorID in openIds:
        print(authorID)
        try:
            all_workids = work_id(authorID)
            print(f'{len(all_workids)}')
            for workID in all_workids:
                work_details = findWork(workID)
                tempList.append(work_details)

        except Exception as e:  # Broader exception handling; replace Exception with more specific exceptions as needed
            print(f"Error processing authorID {authorID}: {str(e)}")
            error_authors.append({'name': name, 'authorID': authorID, 'error': str(e)})
    personDict[name] = tempList  # Storing all work details for each author

print(f'=============ERRORS===============')
if len(error_authors) == 0:
    print(f'+++++++++++NONE++++++++++++')
else:
    for error in error_authors:
        print(error)


with open('openAlex_Output.json', 'w', encoding="utf-8") as jsonFile:
    json.dump(personDict, jsonFile, indent=2)      
       

1. Working to find works by: alan d'andrea
A5045114091
4
A5044648110
682
2. Working to find works by: james fagin
A5080716045
560
A5026026919
33
A5079286906
1
3. Working to find works by: daniel haber
A5038280749
501
A5018290245
35
4. Working to find works by: ethylin jabs
A5035104305
189
A5038991667
152
5. Working to find works by: karl nath
A5034141505
435
6. Working to find works by: bishr omary
A5041239445
27
A5007337339
336
7. Working to find works by: john raymond
A5033763744
264
8. Working to find works by: fuad ziyadeh
A5022475409
200
9. Working to find works by: maria abreu
A5047978655
7
A5012357829
614
10. Working to find works by: michael ackerman
A5055125250
1320
11. Working to find works by: mariana kaplan
A5010407321
330
12. Working to find works by: feroz papa
A5086320327
84
13. Working to find works by: paul taylor
A5078452635
141
14. Working to find works by: loren walensky
A5020222350
178
15. Working to find works by: yiping yang
A5071675435
387
A5019675501
4
16. Work